In [12]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import itertools

from fastprogress import progress_bar

import jlinops

# Functions

In [2]:
def rand_theta(n, num_nsparse, eps=1e-1, reg_lambda=1.0):
    assert num_nsparse < n, "num_nsparse must be less than n."
    result = eps*np.ones(n)
    nsparse_idxs = np.random.choice( [i for i in range(n)], size=num_nsparse, replace=False )
    result[nsparse_idxs] = np.ones(num_nsparse)
    result = result/reg_lambda
    return result

In [30]:
def evaluate_rho(blur_sigma, nsparse, eps, reg_lambda, n_samples=500):

    n = 100
    assert nsparse <= 100, "invalid nsparse."
    F = jlinops.Gaussian1DBlurOperator(n, blur_sigma=blur_sigma)
    F = jlinops.black_box_to_dense(F)
    R, W = jlinops.first_order_derivative_1d(n, boundary="zero")
    _R = R.toarray()
    
    rho_ests = []
    Q1_conds = []
    Q2_conds = []
    for j in range(n_samples):

        theta = rand_theta(n, nsparse, eps, reg_lambda=reg_lambda)

        R = np.diag(1.0/np.sqrt(theta)) @ _R # put in the weights 


        # Get spectrum of original problem
        Q1 = (F.T @ F) + (R.T @ R)
        Q1_eigvals = np.real(np.flip(np.sort(np.linalg.eigvals(Q1))))
        Q1_cond = np.linalg.cond(Q1)
        Q1_conds.append(Q1_cond)

        # Get spectrum for transformed problem
        Rpinv = np.linalg.pinv(R)
        Q2 = np.eye(R.shape[0]) + (Rpinv.T @ F.T @ F @ Rpinv)
        Q2_eigvals = np.real(np.flip(np.sort(np.linalg.eigvals(Q2))))
        Q2_cond = np.linalg.cond(Q2)
        Q2_conds.append(Q2_cond)

        # Improvement factor
        rho = Q1_cond/Q2_cond
        rho_ests.append(rho)

    return np.mean(rho_ests)
        

# Define parameter space

In [40]:
reg_lambdas = np.logspace(-2, 3, 5)
epss = np.logspace(-4, -1, num=4)
nsparses = np.arange(start=0, stop=100+4, step=4)
blur_sigmas = np.arange(start=1, stop=26, step=2)

In [41]:
df = pd.DataFrame({"reg_lambda":[],
              "eps": [],
              "nsparse": [],
              "blur_sigma": [],
              "rho": [],
              })

In [42]:
combinations = itertools.product(reg_lambdas, epss, nsparses, blur_sigmas)
combinations = list(combinations)

for item in progress_bar(combinations):
    reg_lambda, eps, nsparse, blur_sigma = item
    rho = evaluate_rho(blur_sigma, nsparse, eps, reg_lambda)
    df_new = pd.DataFrame({"reg_lambda":[reg_lambda],
              "eps": [eps],
              "nsparse": [nsparse],
              "blur_sigma": [blur_sigma],
              "rho": [rho],
              })
    df = pd.concat([ df, df_new ], ignore_index=True)

df.to_csv("deblurring_gradient_dataset.csv", index=False)

KeyboardInterrupt: 